# Applying functions to a groupby of a pandas dataframe

In this notebook, I'm giving a few examples on applying functions as part of the "groupby" in pandas.

To start, think of the groupby function as the equivalent to a Pivot in Excel. In the Excel, a pivot allows you to summarize data, usually with a **COUNT, SUM** or **MEAN** function.

In pandas, the **groupby** function also allows you to perform some sort of aggregation function on segments or groups in your data.

So, let's first begin by importing the usual necessary libraries:

In [1]:
import pandas
import numpy

Next, for demonstration purposes, we will import a dataset about cars. The **mpg** dataset is a well-known dataset used for exploration and learning purposes. We can find a description of the dataset at the UCI Machine Learning repository: *https://archive.ics.uci.edu/ml/datasets/auto+mpg*. Note that we can download the set from the UCI site, but the dataset does not include headers. We would have to utilize a separate file that has the column names.

So, for ease, we will import this dataset from the seaborn library. You can browse the datasets included in the seaborn package here: *https://github.com/mwaskom/seaborn-data*. But since we don't actually need the seaborn library in this notebook, we will import directly from their github repo's data folder.

In [2]:
file_name = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/mpg.csv"
mpg = pandas.read_csv(file_name)

Let's take a peek at the first few rows to get a feel for what this data set looks like.

In [3]:
mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


The **model_year** is a numeric field, but could also be used as a categoric field. Let's see what the distribution of value counts look like:

In [4]:
mpg.model_year.value_counts()

73    40
78    36
76    34
82    31
75    30
81    29
80    29
79    29
70    29
77    28
72    28
71    28
74    27
Name: model_year, dtype: int64

Likewise, we see that the **cylinders** field is also a discrete numeric field. Let's check that field:

In [5]:
mpg.cylinders.value_counts()

4    204
8    103
6     84
3      4
5      3
Name: cylinders, dtype: int64

The **origin** field is our last categoric field:

In [6]:
mpg.origin.value_counts()

usa       249
japan      79
europe     70
Name: origin, dtype: int64

For our first **groupby** example, we group by **origin**, **model_year**, and **cylinders**, and summarize the **weight** of each group with the **mean** function.

In [7]:
mpg['weight'].groupby([mpg['origin'], mpg['model_year'], mpg['cylinders']]).mean()

origin  model_year  cylinders
europe  70          4            2309.200000
        71          4            2024.000000
        72          4            2573.200000
        73          4            2335.714286
        74          4            2139.333333
        75          4            2571.166667
        76          4            2438.285714
                    6            3820.000000
        77          4            2138.750000
        78          4            2256.666667
                    5            2830.000000
                    6            3275.000000
        79          4            2415.000000
                    5            3530.000000
        80          4            2272.750000
                    5            2950.000000
        81          4            2580.000000
                    6            3160.000000
        82          4            2055.000000
japan   70          4            2251.000000
        71          4            1936.000000
        72          3    

Sometimes we need to apply more than one function to the groups. This can be accomplished with the **agg** function.

This example looks complicated, so let's break it down.

First we specify the columns in the **groupby** function, as we did in the last example: **mpg['origin']**, **mpg['model_year']**, **mpg['cylinders']**. 

Then in the **agg** function, we pass in a dictionary with the field names as the keys, and the functions that we will apply as the values.

Finally, for clarity, we use the **rename** function to give meaningful names to our aggregated columns. Since we have performed different operations on the various columns, we append the function name to the column name. This is just a convention I like to use, so feel free to omit or modify as you like.

In [8]:
mpg.groupby([mpg['origin'], mpg['model_year'], mpg['cylinders']]).agg({
    'name': 'count',
    'weight': 'mean',
    'acceleration': 'median',
    'displacement': 'mean',
    'horsepower': 'mean',
    'mpg': 'median'
}).rename(columns={
    'name': 'name_count', 
    'weight': 'weight_mean',
    'acceleration': 'acceleration_median',
    'displacement': 'displacement_mean',
    'horsepower': 'horsepower_mean',
    'mpg': 'mpg_median'
})

name_count  weight_mean  acceleration_median  \
origin model_year cylinders                                                 
europe 70         4                   5  2309.200000                17.50   
       71         4                   4  2024.000000                16.75   
       72         4                   5  2573.200000                18.00   
       73         4                   7  2335.714286                15.50   
       74         4                   6  2139.333333                15.50   
       75         4                   6  2571.166667                14.75   
       76         4                   7  2438.285714                15.50   
                  6                   1  3820.000000                16.70   
       77         4                   4  2138.750000                14.30   
       78         4                   3  2256.666667                15.70   
                  5                   1  2830.000000                15.90   
                  6                   2  3275.000000                14.70   
       79         4                   3  2415.000000                14.70   
                  5                   1  3530.000000                20.10   
       80         4                   8  2272.750000                16.55   
                  5                   1  2950.000000                19.90   
       81         4                   3  2580.000000                15.80   
                  6                   1  3160.000000                19.60   
       82         4                   2  2055.000000                19.95   
japan  70         4                   2  2251.000000                14.75   
       71         4                   4  1936.000000                16.25   
       72         3                   1  2330.000000                13.50   
                  4                   4  2293.000000                16.00   
       73         3                   1  2124.000000                13.50   
                  4                   2  2329.000000                17.75   
                  6                   1  2807.000000                13.50   
       74         4                   6  2053.000000                17.75   
       75         4                   4  2303.250000                16.50   
       76         4                   3  1980.000000                17.00   
                  6                   1  2930.000000                15.50   
...                                 ...          ...                  ...   
usa    72         4                   5  2263.800000                16.50   
                  8                  13  4228.384615                13.00   
       73         4                   2  2355.500000                19.00   
                  6                   7  2932.857143                16.00   
                  8                  20  4279.050000                12.50   
       74         4                   3  2372.666667                16.50   
                  6                   7  3320.000000                17.00   
                  8                   5  4438.400000                14.50   
       75         4                   2  2615.500000                17.75   
                  6                  12  3398.333333                17.75   
                  8                   6  4108.833333                13.50   
       76         4                   5  2318.200000                17.70   
                  6                   8  3343.250000                17.60   
                  8                   9  4064.666667                13.00   
       77         4                   6  2346.000000                15.85   
                  6                   4  3525.000000                17.30   
                  8                   8  4177.500000                13.10   
       78         4                   6  2417.500000                15.95   
                  6                  10  3322.000000                16.95   
                  8      

On some occasions, we want to use our own custom function, and the **apply** function allows us to do just that.

First, let's create a custom function, called reverse_cumsum. This is a rather contrived example, so just go along with it for the example. 

In [9]:
def reverse_cumsum(x):
    """
    Performs a reverse cumulative sum on a NumPy array
    :param x: list or array
    :return: ndarray
    """
    return numpy.cumsum(x[::-1])[::-1]

Next I'm going to subset the dataset to only 4 cylinder cars from USA.

In [10]:
usa_4cyl = mpg[(mpg.origin=='usa') & (mpg.cylinders==4)]
usa_4cyl.reset_index(inplace=True)
usa_4cyl.head()

,index,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,30,28.0,4,140.0,90.0,2264,15.5,71,usa,chevrolet vega 2300
1,32,25.0,4,98.0,NaN,2046,19.0,71,usa,ford pinto
2,46,22.0,4,140.0,72.0,2408,19.0,71,usa,chevrolet vega (sw)
3,49,23.0,4,122.0,86.0,2220,14.0,71,usa,mercury capri 2000
4,56,26.0,4,91.0,70.0,1955,20.5,71,usa,plymouth cricket


Now we can apply the custom **reverse_cumsum** function to our dataset with the **apply** function. Note that I'm applying this function to the **mpg** and **weight** fields in the dataset.

In [11]:
usa_4cyl[[
    'mpg',
    'weight'
]].groupby([mpg['model_year']]).apply(reverse_cumsum)

mpg   weight
model_year                   
70         0   741.6  66739.0
           1   713.6  64475.0
           2   688.6  62429.0
           3   666.6  60021.0
           4   643.6  57801.0
           5   617.6  55846.0
           6   592.6  53720.0
           7   572.6  51312.0
           8   551.6  49086.0
           9   529.6  46691.0
           10  501.6  44527.0
           11  480.6  42126.0
           12  461.6  39816.0
           13  435.6  37365.0
           14  410.6  34823.0
           15  382.6  32698.0
           16  359.6  30059.0
           17  336.6  27467.0
           18  311.6  24895.0
           19  285.6  22640.0
           20  256.6  20605.0
           21  232.1  18441.0
           22  205.6  15876.0
           23  175.6  13721.0
           24  150.1  11421.0
           25  125.6   8681.0
           26  100.1   5926.0
           27   69.6   3875.0
           28   36.1   1800.0
71         29  813.9  70022.0
...              ...      ...
           42  432.8  38146.0
           43  404.8  35468.0
           44  378.4  32598.0
           45  354.1  29595.0
           46  326.2  26795.0
           47  302.6  23890.0
           48  275.4  21400.0
           49  248.8  18765.0
           50  223.0  16145.0
           51  193.0  13760.0
           52  154.0  11885.0
           53  119.3   9670.0
           54   84.9   7625.0
           55   55.0   5245.0
           56   27.0   2640.0
72         57  449.0  38715.0
           58  415.0  36320.0
           59  384.0  33745.0
           60  355.0  31220.0
           61  328.0  28485.0
           62  304.0  25620.0
           63  281.0  22585.0
           64  243.0  20460.0
           65  207.0  18335.0
           66  181.0  15750.0
           67  145.0  13380.0
           68  118.0  10430.0
           69   91.0   7640.0
           70   59.0   5345.0
           71   31.0   2720.0

[72 rows x 2 columns]

In this article, we demonstrated:
1. how to use the **groupby** function to apply standard functions, 
2. how to use the **agg** function to apply multiple functions to different fields, and
3. how to use the **apply** function to apply a custom function.